# Init

In [2]:
import context
from config import robotConfig
from UR_RTDE_wrapper import RTDEwrapper
print( "Loaded!" )

Loaded: /home/scott/dev_rmstudio/rmlib/rmlib/rmtools Exists?: True
Loaded: /home/scott/dev_rmstudio/rmlib Exists?: True
Loaded!


In [4]:
robotConfig['my_components']['arm_config']['ip_address'] = "192.168.0.101"
rtde = RTDEwrapper( robotConfig['my_components']['arm_config'], None )
print( "Created!" )

Created!


# Test Status

In [3]:
var = rtde.get_status_bits()
print( "get_status_bits:", var, type( var ) )

get_status_bits: 0b11 <class 'str'>


In [4]:
var = rtde.get_safety_status_bits()
print( "get_safety_status_bits:", var, type( var ) )

get_safety_status_bits: 0b1 <class 'str'>


In [5]:
var = rtde.get_tcp_pose_vec()
print( "get_tcp_pose_vec:", var, type( var ) )

get_tcp_pose_vec: [-0.14820359366340688, -0.3121718106510081, 0.1880753162517681, -0.001494138833348034, 3.11879664099641, 0.0376867485511431] <class 'list'>


In [6]:
var = rtde.get_tcp_pose()
print( "get_tcp_pose:\n", var, '\n', type( var ) )

get_tcp_pose:
 [[-0.99422577 -0.10069594  0.03708696 -0.16619304]
 [-0.09964007  0.99459206  0.02930011 -0.42719366]
 [-0.0398368   0.02543557 -0.99888241  0.15037835]
 [ 0.          0.          0.          1.        ]] 
 <class 'numpy.ndarray'>


In [7]:
var = rtde.get_tcp_force()
print( "get_tcp_force:\n", var, '\n', type( var ) )

get_tcp_force:
 [0.5126203959815534, 5.11470410885203, -10.791137039625498, -2.6115225202582675, -0.44625345097893887, -2.130552248130231] 
 <class 'list'>


In [8]:
var = rtde.get_joint_angles()
print( "get_joint_angles:\n", var, '\n', type( var ) )

var = rtde.get_joint_degrees()
print( "get_joint_degrees:\n", var, '\n', type( var ) )

get_joint_angles:
 [0.9448217153549194, -1.6357525030719202, 1.6359248161315918, -1.5246718565570276, -1.5515020529376429, -0.7245577017413538] 
 <class 'list'>
get_joint_degrees:
 [54.13429668214769, -93.72171475398126, 93.73158756505542, -87.35726252309334, -88.89451953920977, -41.51409832347827] 
 <class 'list'>


In [9]:
var = rtde.enable_freedrive()
print( "enable_freedrive:\n", var, '\n', type( var ) )

enable_freedrive:
 True 
 <class 'bool'>


In [10]:
var = rtde.disable_freedrive()
print( "disable_freedrive:\n", var, '\n', type( var ) )

disable_freedrive:
 True 
 <class 'bool'>


# Test Motion

In [3]:
import numpy as np

orig = rtde.get_tcp_pose()
print( "get_tcp_pose:\n", orig )

trgt = orig.copy()
trgt[2,3] += 0.050

print( "New target pose:\n", trgt )

print( "Diff:\n", np.subtract( trgt, orig ) )

rtde.move_speed( trgt, move_type = 'l', speed = 0.125, accel = 0.25,
                 radius = 0, stop_condition = 'dummy', blocking = True )

get_tcp_pose:
 [[-0.99906555  0.03988426  0.01665124 -0.19724808]
 [ 0.0393354   0.99871092 -0.03208159 -0.45727469]
 [-0.01790933 -0.03139663 -0.99934654  0.0759012 ]
 [ 0.          0.          0.          1.        ]]
New target pose:
 [[-0.99906555  0.03988426  0.01665124 -0.19724808]
 [ 0.0393354   0.99871092 -0.03208159 -0.45727469]
 [-0.01790933 -0.03139663 -0.99934654  0.1259012 ]
 [ 0.          0.          0.          1.        ]]
Diff:
 [[0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.05]
 [0.   0.   0.   0.  ]]


True

# Test Condition (Blocking)

## First test without condition

In [4]:
import time
orig = rtde.get_tcp_pose()
trgt = orig.copy()
trgt[2,3] += 0.050

bgn = time.time()
rtde.move_speed( trgt, move_type = 'l', speed = 0.012, accel = 0.125,
                 radius = 0, stop_condition = 'dummy', blocking = True )
print( "Blocking move required", time.time()-bgn , "seconds to execute" )

Blocking move required 4.462336778640747 seconds to execute


## Test with Timeout condition (Blocking)

In [5]:
# Test a condition that belongs to an object, which is often the case
import time
from time import sleep

orig = rtde.get_tcp_pose()
trgt = orig.copy()
trgt[2,3] += 0.100


class OutsideCondition:
    """ Simplest timeout condition """
    
    def __init__( self, TO_s ):
        self.TO  = TO_s
        self.bgn = time.time()
        
    def cond( self ):
        if time.time() - self.bgn >= self.TO:
            print( "TIMEOUT!" )
            return True
        else:
            return False
        

TO = OutsideCondition( 2.0 )

bgn = time.time()
rtde.move_speed( trgt, move_type = 'l', speed = 0.012, accel = 0.125,
                 radius = 0, stop_condition = TO.cond, blocking = True )
print( "Blocking move required", time.time()-bgn , "seconds to execute" )

TIMEOUT!
Blocking move required 2.0767970085144043 seconds to execute


# Test Condition (Non-Blocking)

In [6]:
# Test a condition that belongs to an object, which is often the case
import time
from time import sleep

orig = rtde.get_tcp_pose()
trgt = orig.copy()
trgt[2,3] += 0.100


class OutsideCondition:
    """ Simplest timeout condition """
    
    def __init__( self, TO_s ):
        self.TO  = TO_s
        self.bgn = time.time()
        
    def cond( self ):
        if time.time() - self.bgn >= self.TO:
            print( "TIMEOUT!" )
            return True
        else:
            return False
        
TO = None
TO = OutsideCondition( 2.0 )

bgn = time.time()
rtde.move_speed( trgt, move_type = 'l', speed = 0.012, accel = 0.125,
                 radius = 0, stop_condition = TO.cond, blocking = False )
print( "Blocking move required", time.time()-bgn , "seconds to execute" )

Blocking move required 0.13744091987609863 seconds to execute
TIMEOUT!
COND MET
